# Predicting healthcare coverage from Census data:

####Note: I'll be pulling heavily from here: http://rstudio-pubs-static.s3.amazonaws.com/16587_f28745b007e744f784d5358c843a84e5.html

####Goal: predict the healthcare coverage status (binary variable) based on 29 predictor variables available from the 2009-2011 ''The American Community Survey (ACS) Public Use Microdata Sample (PUMS)'' dataset.

First, let's load the data:

In [131]:
import pandas as pd
import numpy as np

all_data = pd.read_csv('HealthcareData1.txt', sep = '\t')

#What's in here? Use head() function and print out the column headers, and 5-number-summary function:
all_data.head()
print(all_data.columns)
all_data.describe()

Index([u'aa_primarykey', u'agep', u'bld', u'cit', u'cow', u'dis', u'fes',
       u'fs', u'hht', u'hicov', u'hincp', u'indp', u'jwtr', u'mar', u'mil',
       u'msp', u'mv', u'noc', u'np', u'pap', u'puma', u'rac1p', u'rwat',
       u'sch', u'semp', u'sex', u'st', u'type', u'veh', u'wif', u'wkhp'],
      dtype='object')


,aa_primarykey,agep,bld,cit,cow,dis,fes,fs,hht,hicov,...,rac1p,rwat,sch,semp,sex,st,type,veh,wif,wkhp
count,460611.000000,460611.000000,444155.000000,460611.000000,271700.000000,460611.000000,373419.000000,460611.000000,444155.000000,272762.000000,...,460611.000000,444155.000000,444915.000000,376152.000000,460611.000000,460611.000000,460611.000000,444155.000000,373419.000000,235119.000000
mean,230306.000000,39.842518,2.702016,1.410965,2.173121,1.860333,2.963007,1.867305,2.040763,1.115614,...,1.825988,1.003830,1.304872,1779.861234,1.512257,27.643969,1.051251,2.061699,1.599054,37.889864
std,132967.086758,23.420392,1.792557,1.124396,1.947192,0.346641,2.350177,0.339246,1.673337,0.319762,...,2.026162,0.061766,0.549943,14882.866667,0.499850,15.990611,0.282268,1.118627,0.888772,13.199216
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,-9900.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,115153.500000,19.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,12.000000,1.000000,1.000000,1.000000,32.000000
50%,230306.000000,40.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,2.000000,27.000000,1.000000,2.000000,2.000000,40.000000
75%,345458.500000,58.000000,2.000000,1.000000,3.000000,2.000000,4.000000,2.000000,3.000000,1.000000,...,1.000000,1.000000,2.000000,0.000000,2.000000,41.000000,1.000000,3.000000,2.000000,40.000000
max,460611.000000,95.000000,10.000000,5.000000,9.000000,2.000000,8.000000,2.000000,7.000000,2.000000,...,9.000000,2.000000,3.000000,539000.000000,2.000000,56.000000,3.000000,6.000000,3.000000,99.000000


#The hell? What do these columns mean?

Below are the 31 variables in the data set (29 predictors, 1 primary key, 1 outcome variable hicov).

*primary/serial key* (aa_primarykey), *age of person* (agep), *units in structure of home* (bld), *citizen status* (cit), *class of worker* (cow), *disability* (dis), *family type and employment status* (fes), *food stamp/SNAP* (fs), *household/family type* (hht), *health insurance coverage* (hicov), *household income past 12 months* (hincp), *industry type* (indp), *means of transportation to work* (jwtr), *marital status* (mar), *military service* (mil), *married spouse present/absent* (msp), *when moved into house* (mv), *number of children* (noc), *number of person records following this housing record* (np), *public assistance income past twelve months* (pap), *public use microdata area code* (puma), *recoded detailed race code* (rac1), *hot and cold running water* (rwat), *school enrollment* (sch), *self-employment income past 12 months* (semp), *male or female gender* (sex), *state* (st), *housing unit, institutional group quarter, non-institutional quarter* (type), *number of vehicles* (veh), *workers in family during past 12 months* (wif), *usual hours worked per week past 12 months* (wkhp)

####Continuous (or at least, ordinal) variables:
*agep*, *hincp*, *noc*, *np*, *pap*, *semp*, *veh*, *wkhp*

####Categorical (dummy/factor) variables:
*hicov*, *rwat*, *cit*, *fs*, *mar*, *msp*, *puma*, *sex*, *type*, *st*, *dis*, *rac1p*, *hht*, *indp*, *fes*, '*mil*, *sch*, *wif*, *mv*, *cow*, *jwtr*


####Right off the bat:
1. Most of the categorical variables only taking on **binary values** (variables with max == 2 and min == 1) should be changed to 0 and 1, which 0 implying "lack" and 1 implying "presence." Also, we change female == 2 to female == 0.
2. We can **separate the continuous data from the obviously categorical data**, and have Python remember the categorical data as "dtype == 'categorical'"
3. We should **scale** the continuous data (subtract column means and divide by column standard deviations).


In [122]:
cont = ['agep', 'hincp', 'noc', 'np', 'pap', 'semp', 'veh', 'wkhp']
cat = ['hicov', 'rwat', 'cit', 'fs', 'mar', 'msp', 'puma', 'sex', 'type', 'st', 'dis', 'rac1p', 'hht', 'indp', 'fes',
       'mil', 'sch', 'wif', 'mv', 'cow', 'jwtr']

#switch binary 2's to 0's:
replace = ['sex', 'hicov', 'dis', 'fs', 'rwat']
for var in replace:
    all_data[var].iloc[np.where(all_data[var]==2)] = 0

#make categorical variables known as such.
#Must have Pandas 0.15 or newer!!! If not, do
#>pip install pandas --upgrade
cat_data = all_data[cat]
for var in cat:
    cat_data.loc[:,var] = all_data.loc[:,var].astype('category')
    
#scale continuous data: 
from sklearn.preprocessing import scale
cont_data = all_data[cont]
cont_data_mat = cont_data.as_matrix()
for k in range(cont_data_mat.shape[1]):
    col_copy = cont_data_mat[:,k]
    nanloc = np.where(~np.isnan(col_copy))
    m = np.mean(col_copy[nanloc])
    sd = np.std(col_copy[nanloc])
    cont_data_mat[:,k] = (col_copy-m)/sd
    
cont_data = pd.DataFrame(cont_data_mat, columns = cont)

#More Clean-up: removing redunant data, highly correlated variables, and stuff with tons of missing data:


######How to check for lots of missing data, or a ridiculous number of levels:
> all_data['fes'].value_counts(dropna = False) 

> len(all_data['puma'].unique())


######Following along with the link,
1. Let's get rid of state *(st)* because it should coincide with the tons of other socioeconomic data we have
2. Get rid of *indp* because this has 268 different levels, and same with *puma*, which has 630 different levels!
3. Get rid of *fes*, *hht*, and *msp* because they have lots of missing data.
4. Drop *jwtr* because it's missing 259232 entries!
5. Get rid of continuous (or ordinal) data with high correlations (over like 0.8) to prevent collinearity?

In [123]:
#removing columns:
map(cat_data.pop, ['st', 'indp', 'puma', 'fes', 'hht', 'msp', 'type', 'jwtr'])
cor_mat = abs(cont_data.corr())
cor_mat

,agep,hincp,noc,np,pap,semp,veh,wkhp
agep,1.000000,0.043135,0.540040,0.481432,0.006867,0.023214,0.093163,0.100136
hincp,0.043135,1.000000,0.046236,0.140092,0.028383,0.203000,0.293837,0.140335
noc,0.540040,0.046236,1.000000,0.677333,0.027779,0.006562,0.028709,0.009336
np,0.481432,0.140092,0.677333,1.000000,0.018676,0.001581,0.319019,0.027022
pap,0.006867,0.028383,0.027779,0.018676,1.000000,0.002947,0.037515,0.015500
semp,0.023214,0.203000,0.006562,0.001581,0.002947,1.000000,0.033769,0.075373
veh,0.093163,0.293837,0.028709,0.319019,0.037515,0.033769,1.000000,0.003018
wkhp,0.100136,0.140335,0.009336,0.027022,0.015500,0.075373,0.003018,1.000000


##Studying the correlation matrix above,
We see that *np* and *noc* are highly correlated, so we'll remove *noc*. Also, in accordance with the demo, let's remove *semp* too because it measures almost the same thing as *pap*.

In [124]:
cont_data.pop('semp')
cont_data.pop('noc')

data=pd.concat([all_data['aa_primarykey'],cont_data, cat_data], axis = 1)
data.head()

data.to_csv('frank_clean_data.csv', index=False)